In [1]:
import os
import torch, pyro, numpy as np
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from torch import tensor
import torch.nn as nn
import torchvision.transforms.functional as TF

import click


import swyft

DEVICE = 'cuda'

from utils import *

In [6]:
@click.command()
@click.option("--M", type=int, help="Exponent of subhalo mass.")
@click.option("--NSUB", type=int, help="Number of subhaloes.")
@click.option("--NSIM", type=int, default=100, help="Number of simulations to run.")

SyntaxError: unexpected EOF while parsing (506626837.py, line 1)

In [9]:
m = 10
nsub = 3
nsim = 10000

In [10]:
SYSTEM_NAME = "ngc4414"

RUN = f'_m{m}_nsub{nsub}_nsim{nsim}'
SIM_PATH = f'/nfs/scratch/eliasd/store{RUN}.zarr' 

print('run', RUN)


run _m10_nsub3_nsim10000


### Check utilities

In [11]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
CONFIG = get_config(SYSTEM_NAME, str(nsub), str(m))
torch.set_default_tensor_type(torch.FloatTensor)

In [12]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
ppd = CONFIG.ppd()['model_trace'].nodes
torch.set_default_tensor_type(torch.FloatTensor)

In [13]:
ppd['main/sub/m_sub']

{'type': 'sample',
 'name': 'main/sub/m_sub',
 'fn': ExtraIndependent(),
 'is_observed': False,
 'args': (),
 'kwargs': {},
 'value': tensor([1.0000e+10, 1.0000e+10, 1.0000e+10], device='cuda:0'),
 'infer': {},
 'scale': 1.0,
 'mask': None,
 'cond_indep_stack': (),
 'done': True,
 'stop': False,
 'continuation': None}

In [ ]:
all([i == pow(10, m) for i in ppd['main/sub/m_sub']['value']])

In [ ]:
v = ppd['main/sub/p_sub']['value']
sim = simul(v, CONFIG)

print('v', v)
print('sim', sim)

print("sim['image'] min and max" , sim['image'].min(), sim['image'].max() )

# plt.scatter(*v.t(), c="r")
# plt.imshow(simul(v, CONFIG)['image'], **imkwargs)
# plt.colorbar()
# plt.show()

### Simulate

In [ ]:
prior, uv = get_prior(CONFIG)
nx = CONFIG.kwargs["defs"]["nx"]
ny = CONFIG.kwargs["defs"]["ny"]
nsub = CONFIG.umodel.alphas["main"].sub.nsub
print(f'nsub = {nsub}')

In [ ]:
pnames = [f'{z}_{i+1}' for i in range(nsub) for z in ['x', 'y']]
print(pnames)
simulator = swyft.Simulator(model = lambda v: simul(v, CONFIG), 
#                             pnames = ["x_sub", "y_sub"],
                            pnames = pnames,
                            sim_shapes={"image": (nx, ny)})

store = swyft.DirectoryStore(path=SIM_PATH, simulator=simulator)
# store = swyft.MemoryStore(simulator=simulator)

store.add(NSIM, prior)
store.simulate()

In [ ]:
# plt_imshow([p_1s[0] - p_1s[1], p_1s[0] - p_1s[2], p_1s[1] - p_1s[2]], cbar = True, scatter = np.array([np.array(v0s).T]*3), **imkwargs)